In [ ]:
# Code for artery tracking 
#simplified from 1-1

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import json
import cv2
import os
import matplotlib.pyplot as plt
import copy
import numpy as np
import pickle
import glob
import datetime
import pickle


In [ ]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

In [ ]:
# Load CNN Tracker
import sys
sys.path.append(r'U:\LiChen\AICafe\CNNTracker')
from models.centerline_net import CenterlineNet
from centerline_train_tools.data_provider_argu import DataGenerater
from centerline_train_tools.centerline_trainner import Trainer
import torch

In [ ]:
#import iCafe Python
import numpy as np
import sys
#sys.path.append(r'\\DESKTOP2\Ftensorflow\LiChen\iCafe')
sys.path.insert(0,r'\\DESKTOP4\Dtensorflow\LiChen\iCafePython')
from iCafePython import iCafe
from iCafePython import SnakeList,Snake,SWCNode,Point3D

# Load CNNTracker

In [ ]:
#only need to select one model
#Model 1 CNN tracker for ICA TOF MRA
swc_name = 'cnn_snake'

import sys
sys.path.append(r'U:\LiChen\AICafe\CNNTracker')
from models.centerline_net import CenterlineNet

max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)
checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker1-1\classification_checkpoints\centerline_net_model_Epoch_29.pkl"

checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
#Model 2 CNN tracker for Coronary CTA
swc_name = 'cnn_snake'
max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)

checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker2-1\classification_checkpoints\centerline_net_model_Epoch_81.pkl"

checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

In [ ]:
#Model 3 CNN tracker for LATTE
swc_name = 'cnn_snake'
max_points = 500
prob_thr = 0.85

infer_model = CenterlineNet(n_classes=max_points)
checkpoint_path_infer = r"D:\tensorflow\LiChen\AICafe\CNNTracker\CNNTracker4-1\classification_checkpoints\centerline_net_model_Epoch_99.pkl"



checkpoint = torch.load(checkpoint_path_infer)
net_dict = checkpoint['net_dict']
infer_model.load_state_dict(net_dict)
infer_model.to(device)
infer_model.eval()

# Load datasets

In [ ]:
dbname = 'BRAVEAI'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
seg_model_name = 'LumenSeg2-3'

with open(icafe_dir+'/'+dbname+'/db.list','rb') as fp:
    dblist = pickle.load(fp)
train_list = dblist['train']
val_list = dblist['val']
test_list = dblist['test']
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
dbname = 'RotterdanCoronary'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
pilist = ['0_dataset05_U']
seg_model_name = 'CoronarySeg1-8-5'

In [ ]:
dbname = 'UNC'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
seg_model_name = 'LumenSeg5-1'
with open(icafe_dir+'/'+dbname+'/db.list','rb') as fp:
    dblist = pickle.load(fp)
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
dbname = 'HarborViewT1Pre'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
pilist = ['0_ID%d_U'%i for i in [2,9,10,11,12]]
len(pilist)

In [ ]:
# MERGE 
dbname = 'CAREIIMERGEGT'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
seg_model_name = 'LumenSeg6-1'
with open(icafe_dir+'/'+dbname+'/db.list','rb') as fp:
    dblist = pickle.load(fp)
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
dbname = 'IPH-Sup-TOF-FullCoverage'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
seg_model_name = 'LumenSeg7-1'
dblist_name = icafe_dir+'/'+dbname+'/db.list'

with open(dblist_name,'rb') as fp:
    dblist = pickle.load(fp)
        
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist)

In [ ]:
dbname = 'WALLIAI'
icafe_dir = r'\\DESKTOP2\GiCafe\result/'
seg_model_name = 'LumenSeg8-1'
dblist_name = icafe_dir+'/'+dbname+'/db.list'

with open(dblist_name,'rb') as fp:
    dblist = pickle.load(fp)
        
pilist = [pi.split('/')[1] for pi in dblist['test']]
len(pilist),pilist


# Tracking

In [ ]:
# from s.whole.modelname to swc traces
from iCafePython.connect.ext import extSnake
import SimpleITK as sitk

#redo artery tracing
RETRACE = 1
#redo artery tree contraint
RETREE = 1

#segmentation src
seg_src = 's.whole.'+seg_model_name

#Lumen segmentation threshold. 
# Lower value will cause too many noise branches, and neighboring branches will merge as one
# Higher value will reduce the traces detectable
SEGTHRES = 0.5

#max search range in merge/branch, unit in mm
# Higher value will allow larger gap and merge parts of broken arteries, 
#  but will also force noise branches to be merged in the tree
search_range_thres = 10

#which ves to build graph for artery labeling
graph_ves = 'seg_ves_ext_tree2'

DEBUG = 0


for pi in pilist[20:19:-1]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
    if not os.path.exists(icafe_dir+'/'+dbname+'/'+pi):
        os.mkdir(icafe_dir+'/'+dbname+'/'+pi)
        
    icafem = iCafe(icafe_dir+'/'+dbname+'/'+pi)
    
    #select correct version of s.whole from potentially multiple segmentation versions and save as s.whole
    icafem.loadImg(seg_src)
    icafem.saveImg('s.whole',icafem.I[seg_src],np.float16)
    icafem.loadImg('s.whole')

    #export v.tif for 3d visualization if icafe project does not have one already
    if 'v' not in icafem.listAvailImgs():
        vimg = copy.copy(icafem.I['s.whole'])
        vimg[vimg<0] = 0
        vimg = (vimg*255).astype(np.uint16)
        icafem.saveImg('v',vimg,np.int16)
        
    #Tracing
    if RETRACE or not icafem.existPath('seg_ves_ext.swc'):
        if 's.whole' not in icafem.I:
            icafem.loadImg('s.whole')
        seg_ves_snakelist = icafem.constructSkeleton(icafem.I['s.whole']>SEGTHRES)
        
        #load image
        file_name = icafem.getPath('o')
        re_spacing_img = sitk.GetArrayFromImage(sitk.ReadImage(file_name))

        seg_ves_snakelist = icafem.readSnake('seg_ves')
        seg_ves_ext_snakelist = extSnake(seg_ves_snakelist,infer_model,re_spacing_img,DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext',seg_ves_ext_snakelist)
    else:
        seg_ves_ext_snakelist = icafem.readSnake('seg_ves_ext')
        print('read from existing seg ves ext')
    if seg_ves_ext_snakelist.NSnakes==0:
        print('no snake found in seg ves, abort',pi)
        continue
    
    if RETREE or not icafem.existPath('seg_ves_ext_tree.swc'):
        if 's.whole' not in icafem.I:
            icafem.loadImg('s.whole')
        if icafem.xml.res is None:
            icafem.xml.setResolution(0.296875)
            icafem.xml.writexml()
        tree_snakelist = seg_ves_ext_snakelist.tree(icafem,search_range=search_range_thres/icafem.xml.res,int_src='o',DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext_tree', tree_snakelist)
        tree_snakelist = tree_snakelist.tree(icafem,search_range=search_range_thres/3/icafem.xml.res,int_src='s.whole',DEBUG=DEBUG)
        icafem.writeSWC('seg_ves_ext_tree2', tree_snakelist)
        
        tree_main_snakelist = tree_snakelist.mainArtTree(dist_thres=10)
        icafem.writeSWC('seg_ves_ext_tree2_main',tree_main_snakelist)
        
    

# Artery labeling

In [ ]:
from iCafePython.artlabel.artlabel import ArtLabel
art_label_predictor = ArtLabel()

In [ ]:
for pi in pilist[:]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
    if not os.path.exists(icafe_dir+'/'+dbname+'/'+pi):
        os.mkdir(icafe_dir+'/'+dbname+'/'+pi)
        
    icafem = iCafe(icafe_dir+'/'+dbname+'/'+pi)
    
    #generate (simplified node!=2) graph for GNN art labeling
    G = icafem.generateGraph(graph_ves,None,graphtype='graphsim', mode='test', trim=1)
    if len(G.nodes())<5:
        print('too few snakes for artlabeling')
        continue
    icafem.writeGraph(G,graphtype='graphsim')

    #predict landmarks
    pred_landmark, ves_end_pts = art_label_predictor.pred(icafem.getPath('graphsim'),icafem.xml.res)
    #complete graph Gcom for finding the pts in the path
    Gcom = icafem.generateGraph(graph_ves, None, graphtype='graphcom')
    ves_snakelist = findSnakeFromPts(Gcom,G,ves_end_pts)
    print('@@@predict',len(pred_landmark),'landmarks',ves_snakelist)
    #save landmark and ves
    icafem.xml.landmark = pred_landmark
    icafem.xml.writexml()
    icafem.writeSWC('ves_pred', ves_snakelist)

In [ ]:
vimg = vimg[:,:,::-1]

In [ ]:
np.max(vimg)

In [ ]:
icafem.saveImg('v',vimg,np.float16)

In [ ]:
import tifffile
a = tifffile.imread(r"\\DESKTOP2\GiCafe\result\WALLI\47_WALLI-V-09-1-B_M\TH_47_WALLI-V-09-1-B_Mv.tif")

In [ ]:
np.max(a[118])

# Eval

In [ ]:
def eval_simple(snakelist):
    snakelist = copy.deepcopy(snakelist)
    _ = snakelist.resampleSnakes(1)
    #ground truth snakelist from icafem.veslist
    all_metic = snakelist.motMetric(icafem.veslist)
    metric_dict = all_metic.metrics(['MOTA','IDF1','MOTP','IDS'])
    #ref_snakelist = icafem.readSnake('ves')
    snakelist.compRefSnakelist(icafem.vessnakelist)
    metric_dict['OV'], metric_dict['OF'], metric_dict['OT'], metric_dict['AI'], metric_dict['UM'], metric_dict['UMS'], metric_dict['ref_UM'], metric_dict['ref_UMS'], metric_dict['mean_diff'] = snakelist.evalCompDist()
    str = ''
    metric_dict_simple = ['MOTA','IDF1','MOTP','IDS','OV']
    for key in metric_dict_simple:
        str += key+'\t'
    str += '\n'
    for key in metric_dict_simple:
        if type(metric_dict[key]) == int:
            str += '%d\t'%metric_dict[key]
        else:
            str += '%.3f\t'%metric_dict[key]
    print(str)
    return metric_dict

In [ ]:
# calculate metric and save in each pi folder
REFEAT = 0
for pi in pilist[:1]:
    print('='*10,'Start processing',pilist.index(pi),'/',len(pilist),pi,'='*10)
        
    icafem = iCafe(icafe_dir+'/'+pi)
    
    if REFEAT or not icafem.existPath('metric.pickle'):
        print('init metric')
        all_metric_dict = {}
    else:
        print('load metric')
        with open(icafem.getPath('metric.pickle'),'rb') as fp:
            all_metric_dict = pickle.load(fp)
    
    for vesname in ['seg_ves_ext_tree2_main']:
    #for vesname in ['seg_raw','seg_ves_ext_main','seg_ves_ext_tree2']:
    #comparison methods
    #for vesname in ['frangi_ves','seg_unet','seg_raw','raw_sep','cnn_snake','dcat_snake','seg_ves_ext_tree2_main']:
        if vesname in all_metric_dict:
            continue
        print('-'*10,vesname,'-'*10)
        pred_snakelist = icafem.readSnake(vesname)
        if pred_snakelist.NSnakes==0:
            print('no snake',pi,vesname)
            continue
        all_metric_dict[vesname] = eval_simple(pred_snakelist.resampleSnakes(1))
        
    with open(icafem.getPath('metric.pickle'),'wb') as fp:
        pickle.dump(all_metric_dict,fp)
    

In [ ]:
#check feat
pi = pilist[0]
icafem = iCafe(icafe_dir+'/'+pi)
with open(icafem.getPath('metric.pickle'),'rb') as fp:
    all_metric_dict = pickle.load(fp)
all_metric_dict

In [ ]:
#collect feats from pickle
eval_vesname = {'frangi_ves':'Frangi','seg_unet':'U-Net','seg_raw':'DDT',
                'raw_sep':'iCafe','cnn_snake':'CNN Tracker','dcat_snake':'DCAT','seg_ves_ext_tree2_main':'DOST (ours)',
               'seg_ves':'DOST (initial curve)','seg_ves_ext_main':'DOST (deep snake)','seg_ves_ext_tree2':'DOST tree'}
feats = {}
for vesname in eval_vesname:
    feats[vesname] = {}
    
for pi in pilist[:]:
    icafem = iCafe(icafe_dir+'/'+dbname+'/'+pi)
    if not icafem.existPath('metric.pickle'):
        continue
        
    with open(icafem.getPath('metric.pickle'),'rb') as fp:
        all_metric_dict = pickle.load(fp)
    
    #for vesname in all_metric_dict:
    for vesname in eval_vesname:
        if vesname not in all_metric_dict:
            print('no',vesname,'in',pi)
            continue
        for metric in all_metric_dict[vesname]:
            if metric not in feats[vesname]:
                feats[vesname][metric] = []
            feats[vesname][metric].append(all_metric_dict[vesname][metric]) 
    

sel_metrics = ['OV','AI', 'MOTA', 'IDF1', 'IDS']
print('\t'.join(['']+sel_metrics))
for vesname in feats:
    featstr = eval_vesname[vesname]+'\t'
    for metric in sel_metrics:
        if metric in ['IDS']:
            featstr += '%.1f\t'%np.mean(feats[vesname][metric])            
        else:
            featstr += '%.3f\t'%np.mean(feats[vesname][metric])
    print(featstr)